<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [1]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [2]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-03-26 21:55:43.148277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 21:55:43.758453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758496: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import sys
sys.path.append("..")

### Reading Data Files:

In [3]:
from mtsa import calculate_aucroc

2024-03-26 21:55:57.029405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 21:55:57.612061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:57.612103: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:57.612108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [4]:

from mtsa import files_train_test_split
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_train

array([1., 1., 1., 1., 1.])

### MFFCMix Model:

In [5]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [6]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.92

### Hitachi Model:

In [9]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('demux2array', Demux2Array()),
                ('array2melspec',
                 Array2MelSpec(frames=5, hop_length=512, n_fft=1024, n_mels=64,
                               power=2.0, sampling_rate=None)),
                ('final_model',
                 <mtsa.models.hitachi.AutoEncoderMixin object at 0x7f66601473a0>)])

In [8]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 1ms/step


0.24

### RANSynCoders

#### Data preprocessing

In [4]:
from mtsa import Wav2Array

wav_to_array = Wav2Array(mono=False)
input_vector = wav_to_array.transform(X_train)
input_vector_transpose = input_vector.transpose()
input_vector_transpose.shape

(220500, 5)

In [5]:
import numpy as np
indices_linha = np.arange(len(input_vector_transpose), dtype=np.float32).reshape(-1, 1)
indices_linha

array([[0.00000e+00],
       [1.00000e+00],
       [2.00000e+00],
       ...,
       [2.20497e+05],
       [2.20498e+05],
       [2.20499e+05]], dtype=float32)

In [6]:
T_train = np.tile(indices_linha, (1, input_vector_transpose.shape[1]))
T_train

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00],
       [2.00000e+00, 2.00000e+00, 2.00000e+00, 2.00000e+00, 2.00000e+00],
       ...,
       [2.20497e+05, 2.20497e+05, 2.20497e+05, 2.20497e+05, 2.20497e+05],
       [2.20498e+05, 2.20498e+05, 2.20498e+05, 2.20498e+05, 2.20498e+05],
       [2.20499e+05, 2.20499e+05, 2.20499e+05, 2.20499e+05, 2.20499e+05]],
      dtype=float32)

#### Normalization

In [7]:
from sklearn.preprocessing import MinMaxScaler
xscaler = MinMaxScaler()
x_train_scaled = xscaler.fit_transform(input_vector_transpose)
x_train_scaled

array([[0.45243862, 0.45697623, 0.4641472 , 0.47902727, 0.41379306],
       [0.4572944 , 0.45485774, 0.4669805 , 0.46893227, 0.3899437 ],
       [0.4674927 , 0.45469445, 0.45470366, 0.45873913, 0.402269  ],
       ...,
       [0.51886237, 0.4709344 , 0.43578866, 0.44792703, 0.43859902],
       [0.5289    , 0.48571432, 0.42856482, 0.4473591 , 0.43954355],
       [0.50717294, 0.47945595, 0.43226197, 0.45647883, 0.4483705 ]],
      dtype=float32)

In [8]:
N = 10  #5 * round((input_vector_transpose.shape[1] / 3) / 5)  # 10 for both bootstrap sample size and number of estimators
encoder_layers = 1  # number of hidden layers for each encoder
decoder_layers = 2  # number of hidden layers for each decoder
z = int((N / 2) - 1)  # size of latent space
activation = 'relu'
output_activation = 'sigmoid'
S = 5  # Number of frequency components to fit to input signals
delta = 0.05
batch_size = 180
freq_warmup = 5  # pre-training epochs
sin_warmup = 5  # synchronization pre-training
epochs = 10

In [11]:
from mtsa import RANSynCoders

model = RANSynCoders(
    n_estimators=N, 
    max_features=4, 
    encoding_depth=encoder_layers, 
    latent_dim=z, 
    decoding_depth=decoder_layers, 
    activation=activation,
    output_activation=output_activation,
    delta=delta,
    synchronize=False,
    max_freqs=S,
)

In [12]:
model.fit(x_train_scaled, T_train, epochs=epochs, batch_size=batch_size, freq_warmup=freq_warmup, sin_warmup=sin_warmup)


Start of epoch 0
upper_bound_loss: 0.0034623307 lower_bound_loss: 0.0037433212
Start of epoch 1
upper_bound_loss: 0.003682908 lower_bound_loss: 0.0034295316
Start of epoch 2
upper_bound_loss: 0.0031617347 lower_bound_loss: 0.0035936886
Start of epoch 3
upper_bound_loss: 0.0032379045 lower_bound_loss: 0.0034493578
Start of epoch 4
upper_bound_loss: 0.0034687198 lower_bound_loss: 0.003272461
Start of epoch 5
upper_bound_loss: 0.0034595944 lower_bound_loss: 0.0031903828
Start of epoch 6
upper_bound_loss: 0.0032685772 lower_bound_loss: 0.0029391362
Start of epoch 7
upper_bound_loss: 0.0038495008 lower_bound_loss: 0.0035384526
Start of epoch 8
upper_bound_loss: 0.0032675646 lower_bound_loss: 0.0034743175
Start of epoch 9
